In [1]:
import warnings

warnings.filterwarnings(action='ignore')

## Diabetes Dataset

In [2]:
from preprocessing import Dataset
import numpy as np

np.random.seed(227)

test_size = 0.25

n_sample = 2000
target_indices = ['i18400','i27785','i32134','i42916','i45442','i55275','i58296','i64244','i7016','i15937']
n_target = len(target_indices)

Diabetes_entire = Dataset('diabetes')
(x_tra,y_tra), (x_tea,y_tea), (x_taa,y_taa), tra_scaler = Diabetes_entire.split_dataset(test_size, target_indices)

In [3]:
for ti in range(n_target):
    print(ti, target_indices[ti])
    
    x_dis_p = np.load('disinfos/%s_%s_x.npy' % (target_indices[ti], n_sample))
    y_dis_p = np.load('disinfos/%s_%s_y.npy' % (target_indices[ti], n_sample))
    print(y_dis_p.shape)
    if len(y_dis_p.shape) == 2:
        y_dis_p = y_dis_p[:,0]
    
    if ti == 0:
        x_dis, y_dis = x_dis_p, y_dis_p
    else:
        x_dis = np.concatenate((x_dis, x_dis_p), axis=0)
        y_dis = np.concatenate((y_dis, y_dis_p), axis=0).astype(int)

0 i18400
(200, 1)
1 i27785
(200, 1)
2 i32134
(200, 1)
3 i42916
(200,)
4 i45442
(200,)
5 i55275
(200,)
6 i58296
(200,)
7 i64244
(200, 1)
8 i7016
(200, 1)
9 i15937
(200,)


# Insert Disinformation

In [4]:
from model import VictimModels

v_models = VictimModels()
v_models.train_all(x_tra, y_tra)
result_clean = v_models.show_performance([(x_tra,y_tra), (x_tea,y_tea), (x_taa,y_taa)],
                         cnames=['train', 'test','target'])

train models..


100%|██████████| 19/19 [1:18:50<00:00, 249.00s/it]
19it [09:07, 28.82s/it]


In [5]:
x_tr_dis = np.concatenate((x_tra, x_dis), axis=0)
y_tr_dis = np.concatenate((y_tra, y_dis), axis=0).astype(int)

In [6]:
v_models_dis = VictimModels()
v_models_dis.train_all(x_tr_dis, y_tr_dis)
result_dis = v_models_dis.show_performance([(x_tra,y_tra), (x_tea,y_tea), (x_taa,y_taa)],
                         cnames=['train', 'test','target'])

train models..


100%|██████████| 19/19 [6:01:19<00:00, 1141.05s/it]   
19it [10:41, 33.75s/it]


In [16]:
vnames = ['v-svm_sigmoid', 'v-nn_relu_25_10','v-nn_relu_5_2', 'v-svm_poly', 'v-nn_relu_50_25', 'v-svm_rbf']
compare_result(result_clean.loc[vnames], result_dis.loc[vnames])

,mean,std
train acc,-0.080,0.320312
test acc,-0.525,0.639192
target acc,-30.000,18.257419
